# IPU RAY LIBRARY TESTING

This notebook contains instructions to configure, build, run, and test a Poplar C++ Ray/Path Tracer for Graphcore IPUs. This serves to both test the application and to document how test/debug.

## Testing Method

There are automated tests and interactive ones. We will run the automated tests first, then the interactive ones. We recommend you click "run all" in the notebook to execute everything and then read through to understand whether everything is working as expected.

### Build the Code

This notebook assumes you are starting from a clean checkout. The following cell configures and builds everything. The build uses CMake:

In [ ]:
!mkdir -p build
%cd build
!cmake -Wno-dev -G Ninja ..
!ninja -j64

## Automated Tests

First we can check the automated tests are ok:

In [ ]:
!./tests --show_progress

### Test the "trace" Program

Now check we can run the application by rendering a path traced image of the Cornell Box:

In [ ]:
!./trace -w 720 -h 720 --render-mode path-trace --visualise rgb --samples 1000 --ipus 4 --ipu-only --box-only

The output image is high dynamic range (HDR) in EXR format. We can make a function
to perform a quick tone-mapping and display the resulting image in Python:

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Function to apply simple gamma correction, rescale,
# and clip values into range 0-255:
def gamma_correct(x, exposure, gamma):
  scale = 2.0 ** exposure
  y = np.power(x * scale, 1.0 / gamma) * 255.0
  return np.clip(y, 0.0, 255.0)

# Function to plot an opencv image:
def display_image(img):
  plt.figure(figsize=(6, 6))
  plt.style.use('dark_background')
  plt.imshow(cv2.cvtColor(ldr, cv2.COLOR_BGR2RGB), interpolation='bicubic')
  plt.show()

EXR_FLAGS = cv2.IMREAD_UNCHANGED | cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH
hdr = cv2.imread('out_rgb_ipu.exr', EXR_FLAGS)
print(f"HDR image shape: {hdr.shape} type: {hdr.dtype} min: {np.min(hdr)} max: {np.max(hdr)}")

ldr = gamma_correct(hdr, exposure=1.2, gamma=2.4).astype(np.uint8)
cv2.imwrite('tonemapped.png', ldr)
display_image(ldr)

If you want to render a CPU reference image remove the option `--ipu-only` but be aware it will take
much much longer to render. (For a list of all command options run `./test --help`.)

### Comparing AOVs with Embree

We can render the same scene using Embree's CPU ray tracing kernels and compare arbitrary output variables (AOVs). You can also compare AOVs between the IPU and CPU implementations (which use almost identical code). These compariasons demonstrate that basic ray-trace functionality is working and is also useful for debugging when things are broken. For this we use a quicker render-mode `shadow-trace`.

#### Check Normals

For example to compare normals with Embree:

In [ ]:
!./trace -w 1440 -h 1440 --render-mode shadow-trace --visualise normal --ipus 4

Once the outputs are ready we can load them into Python to compare:

In [ ]:
# Load normal images:
ipu_normals = cv2.imread('out_normal_ipu.exr', EXR_FLAGS)
cpu_normals = cv2.imread('out_normal_cpu.exr', EXR_FLAGS)
embree_normals = cv2.imread('out_normal_embree.exr', EXR_FLAGS)

# Function to print simple stats:
def compare(ipu, embree):
  abs_err = np.abs(ipu - embree)
  print(f"IPU min: {np.min(ipu)} max: {np.max(ipu)}")
  print(f"Embree min: {np.min(embree)} max: {np.max(embree)}")
  print(f"ABS Error min: {np.min(abs_err)} max: {np.max(abs_err)} mean: {np.mean(abs_err)}")
  return abs_err

# Function to plot images side by side:
def plot_compare(ipu, embree):
  fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(12, 6))
  ax[0].imshow(ipu)
  ax[0].set_title('IPU')
  ax[1].imshow(embree)
  ax[1].set_title('Embree')
  plt.show()

vis = ((ipu_normals + 1.0) / 2.0)
vis_embree = ((embree_normals + 1.0) / 2.0)
plot_compare(vis, vis_embree)
abs_err = compare(ipu_normals, embree_normals)

We can plot an error histogram (using a log scale because the error counts are small). As you can see most errors are tiny but there are a few outliers - these will be rays that hit alternative (i.e. possibly valid within machine precision) objects due to differences between our intersection test code and Embree's. This scene's dimension range is deliberately large to amplify these differences:

In [ ]:
plt.hist(abs_err.flatten(), bins=300, range=[0.0, np.max(abs_err)], log=True)
plt.show()

#### Check Material Albedo and Shadow Rays

The `--visualise` option allows us to vary the AOV that gets recorded for comparison. For example `rgb` will save albedo and shadow information in `shadow-trace` render mode:

In [ ]:
# Render and load albedo images:
!./trace -w 1440 -h 1440 --render-mode shadow-trace --visualise rgb --ipus 4
ipu_albedo = cv2.cvtColor(cv2.imread('out_rgb_ipu.exr', EXR_FLAGS), cv2.COLOR_BGR2RGB)
cpu_albedo = cv2.cvtColor(cv2.imread('out_rgb_cpu.exr', EXR_FLAGS), cv2.COLOR_BGR2RGB)
embree_albedo = cv2.cvtColor(cv2.imread('out_rgb_embree.exr', EXR_FLAGS), cv2.COLOR_BGR2RGB)

# Plot/compare the albedo:
plot_compare(ipu_albedo, embree_albedo)
abs_err = compare(ipu_albedo, embree_albedo)
plt.hist(abs_err.flatten(), bins=300, range=[0.0, np.max(abs_err)], log=True)
plt.show()

#### Check the Primary Hit-Points

Similarly we can compare the world co-ordinates of the first hits for each pixel:

In [ ]:
# Render and load hit images:
!./trace -w 1440 -h 1440 --render-mode shadow-trace --visualise hitpoint --ipus 4
ipu_hit = cv2.imread('out_hitpoint_ipu.exr', EXR_FLAGS)
cpu_hit = cv2.imread('out_hitpoint_cpu.exr', EXR_FLAGS)
embree_hit = cv2.imread('out_hitpoint_embree.exr', EXR_FLAGS)

# Plot/compare the primary hit points:
ipu_vis = ipu_hit
ipu_vis = (ipu_vis - np.min(ipu_vis)) / (np.max(ipu_vis) - np.min(ipu_vis))
ipu_vis = gamma_correct(ipu_vis, 0, 0.25) / 255
embree_vis = (embree_hit - np.min(embree_hit)) / (np.max(embree_hit) - np.min(embree_hit))
embree_vis = gamma_correct(embree_vis, 0, 0.25) / 255
plot_compare(ipu_vis, embree_vis)

abs_err = compare(ipu_hit, embree_hit)
plt.hist(abs_err.flatten(), bins=300, range=[0.0, np.max(abs_err)], log=True)
plt.show()

### Render the Test Scene (Collada file)

Final check: render a larger scene (should only just fit in tile):

In [ ]:
!./trace -w 720 -h 720 --render-mode path-trace --visualise rgb --samples 2500 --ipus 4 --ipu-only --mesh-file ../assets/test_scene.dae --load-normals

In [ ]:
hdr = cv2.imread('out_rgb_ipu.exr', EXR_FLAGS)
ldr = gamma_correct(hdr, exposure=-0.5, gamma=2.6).astype(np.uint8)
display_image(ldr)